In [25]:
import yfinance as yf
import time
import pandas as pd
import json
import numpy as np
import xgboost
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier
import warnings

warnings.filterwarnings('ignore')


In [2]:
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split, cross_val_score
# from sklearn.datasets import load_boston
from sklearn.metrics import mean_squared_error

In [3]:
hedging_pnl = pd.read_csv('Options constructor dummy - raw_hedging_pnls.csv')
options_pnl = pd.read_csv('Options constructor dummy - raw_options_pnls.csv')

hedging_pnl['0'] = pd.to_datetime(hedging_pnl['0'])
options_pnl['0'] = pd.to_datetime(options_pnl['0'])

options_pnl = options_pnl.set_index('0').astype(float)
hedging_pnl = hedging_pnl.set_index('0').astype(float)

base_features = pd.read_csv('new_features.csv')
base_features['Unnamed: 0'] = pd.to_datetime(base_features['Unnamed: 0'])
base_features = base_features.set_index('Unnamed: 0')
base_features = base_features.resample('W-SUN').sum()

strats = list(hedging_pnl.columns)

In [4]:
strats_data = {}
for i in strats:
    strat_info = (options_pnl[i]+hedging_pnl[i])
    weekly_pnl = strat_info.resample('W-SUN').sum()

    strats_data.update({i:weekly_pnl})

In [89]:
strats_data

{'0.01_10_15_0_put': 0
 2019-10-20    0.100000
 2019-10-27    0.125000
 2019-11-03    0.225000
 2019-11-10    1.405095
 2019-11-17    0.000000
                 ...   
 2023-05-07    0.000000
 2023-05-14    0.000000
 2023-05-21    0.000000
 2023-05-28    0.000000
 2023-06-04    0.000000
 Freq: W-SUN, Name: 0.01_10_15_0_put, Length: 190, dtype: float64,
 '0.01_10_15_0_call': 0
 2019-10-20    0.0500
 2019-10-27   -0.4215
 2019-11-03    0.0750
 2019-11-10    0.0250
 2019-11-17    0.0000
                ...  
 2023-05-07    0.0000
 2023-05-14    0.0000
 2023-05-21    0.0000
 2023-05-28    0.0000
 2023-06-04    0.0000
 Freq: W-SUN, Name: 0.01_10_15_0_call, Length: 190, dtype: float64,
 '0.03_10_15_0_put': 0
 2019-10-20   -1.027405
 2019-10-27    0.475000
 2019-11-03   -0.237405
 2019-11-10    0.500000
 2019-11-17    0.000000
                 ...   
 2023-05-07    0.000000
 2023-05-14    0.000000
 2023-05-21    0.000000
 2023-05-28    0.000000
 2023-06-04    0.000000
 Freq: W-SUN, Name: 0.03_

In [5]:
for strat in strats:
    
    str_data = pd.DataFrame()
    str_data['value'] = strats_data[strats[100]]
    str_data['target'] = str_data['value'].shift(-1)
    
    
    df = pd.merge(str_data, base_features, how = 'inner', right_index = True, left_index = True).dropna()
    df['target'] = np.where(df['target']>=0,1,0)

    features = df.drop('target', axis = 1)
    target = df['target']

    break

In [6]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3)

In [85]:
X_train

,value,^GSPC,^VIX,^VIX9D,^VVIX,Net Gamma,rv5,rv10
2023-05-07,1.286830,-0.007715,0.107298,0.033187,0.061162,-2.367714,-0.011343,0.141579
2023-03-26,0.716995,0.014062,-0.149608,-0.254093,-0.212416,683.304983,-0.191057,-0.341238
2023-03-19,2.445485,0.014529,0.054818,0.056941,0.169112,-1.031613,-0.383220,0.129777
2023-02-12,-6.334200,-0.010983,0.118691,0.211717,0.143642,-1.451010,-0.516659,-0.205946
2023-02-19,2.225000,-0.002615,-0.016538,-0.189497,-0.079953,3.842687,0.025495,-0.380886
2023-04-09,1.200000,-0.001011,-0.015192,-0.047392,-0.032531,-0.792282,-1.116451,0.163595
2023-02-26,1.647885,-0.026823,0.090475,0.114303,-0.023780,-12.509335,-0.073370,0.994995
2023-03-05,1.200000,0.019042,-0.155212,-0.030475,-0.114170,0.685980,1.828620,-0.867823
2023-04-16,1.194620,0.008013,-0.071278,-0.149293,-0.006866,4.176810,1.112414,-1.038857
2023-04-23,0.170875,-0.000972,-0.016256,0.114321,0.148636,-0.607816,0.069561,0.207701


In [7]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)

In [8]:
y_test

array([1, 0, 1, 1, 1, 0], dtype=int64)

In [9]:
xgb = XGBClassifier(
                   learning_rate = 0.02,
                   n_estimators = 100,
                   max_depth = 3,
                   min_child_weight = 1,
                   subsample = 0.75,
                   colsample_bytree = 0.6,
                   gamma = 0)


# Fitting the model on the training data
xgb.fit(X_train, y_train)

xgb.save_model('xgb_last.bin')  # or 'model_name.model'


C:\Users\Alexander\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [17:35:01] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)


In [165]:
def objective(trial):
    param = {
        'tree_method': 'hist',  # Fast method for large datasets
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'booster': trial.suggest_categorical('booster', ['gbtree', 'gblinear', 'dart']),
        'lambda': trial.suggest_loguniform('lambda', 1e-8, 1.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-8, 1.0),
        'subsample': trial.suggest_uniform('subsample', 0.4, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.4, 1.0),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-8, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10)
    }

    model = XGBClassifier(**param)
    model.fit(X_train, y_train)

    preds = model.predict(X_test)

    rmse = mean_squared_error(y_test, preds, squared=False)

    return rmse

In [27]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

print("Best hyperparameters: ", study.best_params)

[I 2024-09-01 17:35:59,559] A new study created in memory with name: no-name-37ca9eb4-a0ed-4d1d-9b26-269c395d4556
[I 2024-09-01 17:35:59,649] Trial 0 finished with value: 0.5773502691896257 and parameters: {'booster': 'gblinear', 'lambda': 4.898866673568418e-08, 'alpha': 0.18163560225108555, 'subsample': 0.8547470742026353, 'colsample_bytree': 0.8793776406520789, 'max_depth': 9, 'n_estimators': 776, 'learning_rate': 4.486053365027686e-07, 'min_child_weight': 8}. Best is trial 0 with value: 0.5773502691896257.
[I 2024-09-01 17:35:59,685] Trial 1 finished with value: 0.5773502691896257 and parameters: {'booster': 'gblinear', 'lambda': 4.841184182117036e-08, 'alpha': 3.219215488245197e-07, 'subsample': 0.9328421942252693, 'colsample_bytree': 0.9760670024955787, 'max_depth': 4, 'n_estimators': 158, 'learning_rate': 3.5589005584903616e-07, 'min_child_weight': 1}. Best is trial 0 with value: 0.5773502691896257.
[I 2024-09-01 17:35:59,836] Trial 2 finished with value: 0.5773502691896257 and p

Best hyperparameters:  {'booster': 'gblinear', 'lambda': 4.898866673568418e-08, 'alpha': 0.18163560225108555, 'subsample': 0.8547470742026353, 'colsample_bytree': 0.8793776406520789, 'max_depth': 9, 'n_estimators': 776, 'learning_rate': 4.486053365027686e-07, 'min_child_weight': 8}


In [29]:
best_params = study.best_params

model = XGBClassifier(**best_params)
model.fit(X_train, y_train)

preds = model.predict(X_test)
rmse = mean_squared_error(y_test, preds, squared=False)
print(f"RMSE: {rmse}")

RMSE: 0.5773502691896257


In [111]:
df_ff1 = pd.read_csv("QC Verification VI - 22_logs.csv").dropna()
df_ff2 = pd.read_csv("QC Verification VI - 23-24_logs.csv").dropna()
df_ff = pd.concat([df_ff1, df_ff2])

In [115]:
df_ff['Time_Action'] = pd.to_datetime(df_ff['Time_Action'])
df_ff = df_ff.set_index('Time_Action')

In [127]:
df_ff['PnL'] = (df_ff['Daily_running_HEDGE_pnl']+df_ff['Option_pnl']).astype(float)

In [129]:
df_ff['SPY_Price_diff'] = df_ff['SPY_Price'].astype(float).diff()
df_ff['VIX_Price_diff'] = df_ff['VIX_Price'].astype(float).diff()

In [137]:
df_ff['target'] = df_ff['PnL'].shift(-1)

In [147]:
data_df = pd.DataFrame()
for i in range(30):
    for feature in ['Options_Delta', 'SPY_Price_diff', 'VIX_Price_diff', 'PnL']:
        data_df[f'{feature}_{i}'] = df_ff[feature].shift(i)

In [151]:
data_df['target'] = df_ff['target']

In [157]:
data_df = data_df.dropna()

In [161]:
X_train, X_test, y_train, y_test = train_test_split(data_df.drop('target', axis = 1), data_df['target'], test_size=0.3)

In [163]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

print("Best hyperparameters: ", study.best_params)

[I 2024-09-02 09:11:56,304] A new study created in memory with name: no-name-93a1b907-282a-4bd4-8e62-3118d59c51ed
